In [1]:
import numpy as np
import os
FLUX_DATA_DIR = 'C:\\Users\\Alberto\\Desktop\\uni_tmp\\ADL\\STACIE\\flux_prediction\\data\\magnetic_flux_area_data\\'
LSTM_DATA_DIR = 'C:\\Users\\Alberto\\Desktop\\uni_tmp\\ADL\\STACIE\\flux_prediction\\data\\lstm_data\\multi_label_classification\\'

In [2]:
file_inp = LSTM_DATA_DIR + 'train_input.npy'
file_tar = LSTM_DATA_DIR + 'train_target.npy'

train_input = np.load(file_inp)
train_target = np.load(file_tar)

file2_inp = LSTM_DATA_DIR + 'val_input.npy'
file2_tar = LSTM_DATA_DIR + 'val_target.npy'

val_input = np.load(file2_inp)
val_target = np.load(file2_tar)

In [28]:
t2_input = train_input
t2_output = train_target

In [29]:
my_shape = t2_input.shape
#my_shape = (my_shape[0], my_shape[1], my_shape[2] + 5)
my_shape2 = t2_output.shape

print(my_shape, my_shape2)

(77738, 10, 40) (77738, 1, 5)


In [30]:
array1 = np.pad(t2_input, ((0,0), (0,0), (0,5)), mode='constant')
array2 = np.pad(t2_output, ((0,0), (0,0), (40,0)), mode='constant')

learning rate tutorial: https://pytorch-forecasting.readthedocs.io/en/stable/tutorials/stallion.html

In [34]:
array2[500][0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.])

In [27]:
print(train_input[0][0])
print(array1[0][0])

[-0.56783394  0.         -0.12532183  0.         -0.71248128 -0.43924826
  1.35573235 -0.57862741  0.         -0.64730654  0.15149191  0.
  0.         -0.13710701 -0.15285074  0.58515245  0.          0.49902362
  0.          3.8277182   1.20383338 -0.13714851  0.33179481  3.44528444
 -1.35885841  0.24234076  1.3449445   0.74412219  0.          0.
  0.          0.          0.          0.53544267  0.71633556  1.36444972
 -0.62381646  0.          0.          0.        ]
[-0.56783394  0.         -0.12532183  0.         -0.71248128 -0.43924826
  1.35573235 -0.57862741  0.         -0.64730654  0.15149191  0.
  0.         -0.13710701 -0.15285074  0.58515245  0.          0.49902362
  0.          3.8277182   1.20383338 -0.13714851  0.33179481  3.44528444
 -1.35885841  0.24234076  1.3449445   0.74412219  0.          0.
  0.          0.          0.          0.53544267  0.71633556  1.36444972
 -0.62381646  0.          0.          0.          0.          0.
  0.          0.          0.        ]


In [6]:
train_input.shape

(77738, 10, 40)

In [5]:
train_target.shape

(77738, 1, 5)

In [36]:
len(train_target[0][0])

5

In [11]:
lstm_src = np.load(os.path.join(LUIS_DATA_DIR, 'test_input.npy'))
lstm_trg = np.load(os.path.join(LUIS_DATA_DIR, 'test_target.npy'))

In [15]:
len(lstm_src[0][0])

40

In [17]:
lstm_trg[0]

array([[0., 1., 0., 0., 0.]])

In [3]:
sources = np.load(os.path.join(FLUX_DATA_DIR, 'set_2', 'input_res_concatenated.npy'))
targets = np.load(os.path.join(FLUX_DATA_DIR, 'set_2', 'target_res_concatenated.npy'))